In [1]:
import requests
import pandas as pd 
import json
import numpy as np
import zipfile

## Function to Process Data

In [2]:
def process_census_data(data,zipcodes_file):
    """
    Process census data to extract median income and housing costs for specific zip codes.
    
    Parameters:
        csv_file (str): Path to the census data CSV file.
        zipcodes_file (str): Path to the file containing zip codes to filter.
    
    Returns:
        pd.DataFrame: Processed DataFrame with relevant zip codes, median income, and housing costs.
    """
    
    # Extract zip codes and clean
    data['NAME'] = data['NAME'].str[5:11].str.strip()
    data = data.iloc[1:, :]
    data['NAME'] = data['NAME'].astype(int)
    
    # Load zip codes to filter
    with open(zipcodes_file, "r") as file:
        zipcodes_to_filter_out = {int(line.strip()) for line in file}
    
    # Filter the data
    df_filtered = data[data['NAME'].isin(zipcodes_to_filter_out)]
    
    # Select relevant columns and rename
    df_final = df_filtered[['NAME', 'S2503_C01_013E', 'S2503_C01_024E']]
    df_final.rename(columns={'NAME': 'zipcode',
                              'S2503_C01_013E': 'median_income',
                              'S2503_C01_024E': 'median_housing_costs_monthly'}, inplace=True)
    
    # Replace missing values and convert data types
    df_final.replace("-", 0, inplace=True)
    df_final['median_income'] = df_final['median_income'].astype(int)
    
    # Clean up housing costs
    df_final['median_housing_costs_monthly'] = df_final['median_housing_costs_monthly'].str.replace(r'\+$', '', regex=True)
    df_final['median_housing_costs_monthly'] = df_final['median_housing_costs_monthly'].str.replace(',', '', regex=True)
    df_final['median_housing_costs_monthly'] = df_final['median_housing_costs_monthly'].fillna(0).astype(int)
    
    # Calculate yearly housing costs
    df_final['yearly_housing_costs'] = df_final['median_housing_costs_monthly'] * 12
    
    return df_final


## Function to Load Data

In [3]:
import requests

def get_data(url):
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])  # First row contains column names
    return df


data_2023= get_data("https://api.census.gov/data/2023/acs/acs5/subject?get=group(S2503)&ucgid=pseudo(0400000US06$8600000)")


In [4]:
data_2022= get_data("https://api.census.gov/data/2022/acs/acs5/subject?get=group(S2503)&ucgid=pseudo(0400000US06$8600000)")
data_2021=get_data("https://api.census.gov/data/2021/acs/acs5/subject?get=group(S2503)&ucgid=pseudo(0400000US06$8600000)")
data_2020=get_data("https://api.census.gov/data/2020/acs/acs5/subject?get=group(S2503)&ucgid=pseudo(0400000US06$8600000)")
data_2019=get_data("https://api.census.gov/data/2019/acs/acs5/subject?get=group(S2503)&ucgid=pseudo(0400000US06$8600000)")
data_2018=get_data("https://api.census.gov/data/2018/acs/acs5/subject?get=group(S2503)&ucgid=pseudo(0400000US06$8600000)")

In [7]:
data_2022_copy=data_2022.copy()
data_2021_copy=data_2021.copy()
data_2020_copy=data_2020.copy()
data_2019_copy=data_2019.copy()
data_2018_copy=data_2018.copy()

## Function to Join 2023 Zip Codes to the rest of the Years

In [8]:
def fill_zipcodes(data_2023,data):
    name_mapping = dict(zip(data_2023['GEO_ID'], data_2023['NAME']))
    data['NAME'] = data['GEO_ID'].map(name_mapping).fillna(data['NAME'])
    return data

In [9]:
data_2022_zip=fill_zipcodes(data_2023,data_2022_copy)
data_2021_zip=fill_zipcodes(data_2023,data_2021_copy)
data_2020_zip=fill_zipcodes(data_2023,data_2020_copy)
data_2019_zip=fill_zipcodes(data_2023,data_2019_copy)
data_2018_zip=fill_zipcodes(data_2023,data_2018_copy)


In [10]:
data_2023_processed= process_census_data(data_2023,'LAZipCodes.txt')
data_2022_processed= process_census_data(data_2022_zip,'LAZipCodes.txt')
data_2021_processed= process_census_data(data_2021_zip,'LAZipCodes.txt')
data_2020_processed= process_census_data(data_2020_zip,'LAZipCodes.txt')
data_2019_processed= process_census_data(data_2019_zip,'LAZipCodes.txt')
data_2018_processed= process_census_data(data_2018_zip,'LAZipCodes.txt')

C:\Users\User\AppData\Local\Temp\ipykernel_3944\1860716874.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['NAME'] = data['NAME'].astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_3944\1860716874.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.rename(columns={'NAME': 'zipcode',
C:\Users\User\AppData\Local\Temp\ipykernel_3944\1860716874.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [14]:

data_2023_processed= data_2023_processed.rename(columns={'median_income':'median_income_2023',
                                             'median_housing_costs_monthly':'median_housing_costs_monthly_2023',
                                             'yearly_housing_costs':'yearly_housing_costs_2023'})

data_2022_processed= data_2022_processed.rename(columns={'median_income':'median_income_2022',
                                             'median_housing_costs_monthly':'median_housing_costs_monthly_2022',
                                             'yearly_housing_costs':'yearly_housing_costs_2022'})

data_2021_processed= data_2021_processed.rename(columns={'median_income':'median_income_2021',
                                             'median_housing_costs_monthly':'median_housing_costs_monthly_2021',
                                             'yearly_housing_costs':'yearly_housing_costs_2021'})

data_2020_processsed= data_2020_processed.rename(columns={'median_income':'median_income_2020',
                                             'median_housing_costs_monthly':'median_housing_costs_monthly_2020',
                                             'yearly_housing_costs':'yearly_housing_costs_2020'})


data_2019_processsed= data_2019_processed.rename(columns={'median_income':'median_income_2019',
                                             'median_housing_costs_monthly':'median_housing_costs_monthly_2019',
                                             'yearly_housing_costs':'yearly_housing_costs_2019'})

data_2018_processsed= data_2018_processed.rename(columns={'median_income':'median_income_2018',
                                             'median_housing_costs_monthly':'median_housing_costs_monthly_2018',
                                             'yearly_housing_costs':'yearly_housing_costs_2018'})

In [17]:
from functools import reduce

dfs = [data_2018_processsed, data_2019_processsed, data_2020_processsed, data_2021_processed, data_2022_processed, data_2023_processed]

# Perform an inner join on "zipcode"
final_df = reduce(lambda left, right: pd.merge(left, right, on="zipcode", how="inner"), dfs)


In [18]:
final_df

,zipcode,median_income_2018,median_housing_costs_monthly_2018,yearly_housing_costs_2018,median_income_2019,median_housing_costs_monthly_2019,yearly_housing_costs_2019,median_income_2020,median_housing_costs_monthly_2020,yearly_housing_costs_2020,median_income_2021,median_housing_costs_monthly_2021,yearly_housing_costs_2021,median_income_2022,median_housing_costs_monthly_2022,yearly_housing_costs_2022,median_income_2023,median_housing_costs_monthly_2023,yearly_housing_costs_2023
0,90001,38521,1164,13968,43360,1221,14652,48011,1245,14940,52806,1301,15612,57698,1413,16956,60751,1493,17916
1,90002,35410,1254,15048,37285,1301,15612,42245,1396,16752,46159,1417,17004,54221,1558,18696,56158,1648,19776
2,90003,37226,1261,15132,40598,1304,15648,42220,1352,16224,47733,1396,16752,51275,1503,18036,54781,1609,19308
3,90004,48754,1296,15552,49675,1371,16452,52775,1469,17628,54947,1591,19092,58420,1767,21204,62655,1847,22164
4,90005,35149,1166,13992,38491,1264,15168,42398,1332,15984,44913,1446,17352,49226,1596,19152,52755,1651,19812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,93551,92500,1849,22188,93055,1854,22248,93637,1941,23292,98181,2023,24276,107133,2266,27192,110850,2366,28392
291,93552,63261,1514,18168,65728,1582,18984,66289,1586,19032,73074,1706,20472,85890,1889,22668,84894,1931,23172
292,93553,57833,1120,13440,51630,1294,15528,62875,1283,15396,59805,1316,15792,71587,1581,18972,73029,1209,14508
293,93563,82656,1082,12984,-666666666,1191,14292,200625,2023,24276,210600,2120,25440,-666666666,2710,32520,-666666666,2710,32520
